In [1]:
#auto reload modules
%load_ext autoreload
%autoreload 2

from datetime import datetime
import os
import random
import json
from src.llm.PredictionUtils import init_model_and_tokenizer,llm_gen
from src.prop.utils import gen_reason,gen_masked_prediction_problem_prompts,parse_prediction_with_check

pid = os.getpid()
seed = int(pid)+int(datetime.now().timestamp())
print("seed: ", seed)
random.seed(seed)



out_dir = "data"
os.system(f"mkdir -p {out_dir}")

current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")



seed:  1727393140


In [2]:
model_id="meta-llama/Meta-Llama-3.1-70B-Instruct"
model,tokenizer=init_model_and_tokenizer(model_id)

INFO 09-19 15:17:47 config.py:904] Defaulting to use mp for distributed inference
INFO 09-19 15:17:47 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='meta-llama/Meta-Llama-3.1-70B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=8, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llam

Loading safetensors checkpoint shards:   0% Completed | 0/30 [00:00<?, ?it/s]


(VllmWorkerProcess pid=666600) INFO 09-19 15:18:07 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=666603) INFO 09-19 15:18:07 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=666601) INFO 09-19 15:18:07 model_runner.py:1008] Loading model weights took 16.4605 GB
INFO 09-19 15:18:08 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=666606) INFO 09-19 15:18:08 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=666604) INFO 09-19 15:18:08 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=666602) INFO 09-19 15:18:09 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=666605) INFO 09-19 15:18:09 model_runner.py:1008] Loading model weights took 16.4605 GB
INFO 09-19 15:18:13 distributed_gpu_executor.py:57] # GPU blocks: 82648, # CPU blocks: 6553
(VllmWorkerProcess pid=666602) INFO 09-19 15:18:15 

In [3]:
from datasets import load_dataset

ds=load_dataset("kanhatakeyama/material-properties",split="Bradley")
ds=ds.shuffle(seed=1)
len(ds)

README.md:   0%|          | 0.00/560 [00:00<?, ?B/s]

Bradley-00000-of-00001.parquet:   0%|          | 0.00/872k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7241 [00:00<?, ? examples/s]

Generating Bradley split:   0%|          | 0/24889 [00:00<?, ? examples/s]

24889

In [4]:
train_ds=ds.select(range(24000))
test_ds=ds.select(range(24000,24800))
train_ds[0]

{'CompName': '(2Z)-2-(1,3-Benzodioxol-5-ylmethylene)-6,7-dihydro-5H-[1,3]thiazolo[3,2-a]pyrimidin-3(2H)-one',
 'SMILES': 'O=C1C(SC2=NCCCN12)=Cc4ccc3OCOc3c4',
 'Property': 'Melting temperature',
 'Value': 151.35,
 'unit': '[oC]',
 'Source': 'BradleyMeltingPointDataset'}

In [5]:
from tqdm import tqdm
n_records=500

generated_records=[]
for _ in tqdm(range(10**3)):
    problems=[]
    prompt_list=[
    ]
    for i in range(n_records):
        record=random.choice(train_ds)
        #prompt,actual_value=gen_problem(record,tokenizer)
        prompt,actual_value=gen_reason(record,tokenizer)
        problems.append(
            {
            "record":record,
            "prompt":prompt,
            "actual_value":actual_value   
            }

        )
        prompt_list.append(prompt)

    #interference
    predicted_text_list=llm_gen(model,prompt_list)
    
    #predict properties according to the generated reasonings.
    check_prompt_list,masked_reason_list=gen_masked_prediction_problem_prompts(predicted_text_list,problems,tokenizer)
    predicted_value_list=llm_gen(model,check_prompt_list)


    #parse result
    #prediction_records=parse_prediction(problems,predicted_text_list)
    prediction_records=parse_prediction_with_check(problems, masked_reason_list, predicted_value_list)

    generated_records.extend(prediction_records)
    for i in range(len(prediction_records)):
        with open(f"data/{current_time_no_symbols}_llm_gen.jsonl","a", encoding='utf-8') as f:
            f.write(json.dumps(prediction_records[i],ensure_ascii=False)+"\n")

  0%|          | 0/1000 [00:00<?, ?it/s]

  5%|▍         | 46/1000 [59:38<20:36:47, 77.79s/it]


KeyboardInterrupt: 